In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def scrape_site(url):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return soup

In [21]:
teams_url = r'https://www.2kratings.com/current-teams/'

soup_teams = scrape_site(teams_url)
teams_raw = soup_teams.body.find_all('tbody')[0].find_all('tr')

teams = []
for team in teams_raw:
    teams.append([team.a['href'], str(team.a.text).strip()])
teams

[['https://www.2kratings.com/teams/atlanta-hawks', 'Atlanta Hawks'],
 ['https://www.2kratings.com/teams/boston-celtics', 'Boston Celtics'],
 ['https://www.2kratings.com/teams/brooklyn-nets', 'Brooklyn Nets'],
 ['https://www.2kratings.com/teams/charlotte-hornets', 'Charlotte Hornets'],
 ['https://www.2kratings.com/teams/chicago-bulls', 'Chicago Bulls'],
 ['https://www.2kratings.com/teams/cleveland-cavaliers',
  'Cleveland Cavaliers'],
 ['https://www.2kratings.com/teams/dallas-mavericks', 'Dallas Mavericks'],
 ['https://www.2kratings.com/teams/denver-nuggets', 'Denver Nuggets'],
 ['https://www.2kratings.com/teams/detroit-pistons', 'Detroit Pistons'],
 ['https://www.2kratings.com/teams/golden-state-warriors',
  'Golden State Warriors'],
 ['https://www.2kratings.com/teams/houston-rockets', 'Houston Rockets'],
 ['https://www.2kratings.com/teams/indiana-pacers', 'Indiana Pacers'],
 ['https://www.2kratings.com/teams/los-angeles-clippers',
  'Los Angeles Clippers'],
 ['https://www.2kratings.co

In [37]:
data = {}
for team in teams:
    team_url = team[0]
    team_name = team[1]
    
    soup_roster = scrape_site(team_url)
    roster_raw = soup_roster.find_all('tbody')[0].find_all('span', class_='entry-font')
    
    roster = []
    for player in roster_raw:
        player_url = player.a['href']
        player_name = player.a.text
        roster.append([r'%s' % str(player_url), str(player_name)])
        
    for player in roster:
        player_team = team_name
        player_name = player[1]
        player_url = player[0]
        
        per_player_dict = {}
        
        soup_attributes = scrape_site(player_url)
        overall_rating = soup_attributes.find_all('span', class_='attribute-box-player')[0].text
        
        per_player_dict['scrape_date'] = pd.to_datetime('today').strftime('%Y-%m-%d')
        per_player_dict['Team'] = player_team
        per_player_dict['Name'] = player_name
        per_player_dict['Overall Rating'] = overall_rating
        
        player_info = soup_attributes.find_all('div', class_='header-subtitle')[0].find_all('p', class_='mb-0')
        for info in player_info:
            text = info.text
            if 'Nationality' in text:
                per_player_dict['Nationality'] = text.split(':')[-1].strip()
            if 'Archetype' in text:
                per_player_dict['Archetype'] = text.split(':')[-1].strip()
            if 'Position' in text:
                per_player_dict['Position'] = text.split(':')[-1].strip()
            if 'Height' and 'Weight' in text:
                height, weight = text.split('|')[0].strip().split(':')[-1].strip(), text.split('|')[-1].split(':')[-1].strip()
                per_player_dict['Height'] = height
                per_player_dict['Weight'] = weight
            if 'Jersey' in text:
                per_player_dict['Jersey'] = text.split('#')[-1].strip()
            if 'in the NBA' in text:
                per_player_dict['Years in NBA'] = text.split(':')[-1].strip()
            if 'Prior to' in text:
                per_player_dict['Prior to NBA'] = text.split(':')[-1].strip()
        
        attributes_raw = soup_attributes.find_all('div', class_='row mr-md-n4')[0].find_all('li')
        for attribute in attributes_raw:
            text = attribute.text
            skill_name = text[3:]
            skill_level = text[:2]
            per_player_dict[skill_name] = skill_level
        
        badge_figure_rawer = soup_attributes.find_all('div', class_='badge-card')
        badge_figure_raw = badge_figure_rawer[:int(len(badge_figure_rawer)/2)] # remove duplicates

        badges = {}
        for badge_raw in badge_figure_raw:
            name = badge_raw.find_all('h4')[0].text    
            category = badge_raw.find_all('span', class_='badge-pill')[0].text
            description = badge_raw.find_all('p', class_='badge-description')[0].text
            level_raw = badge_raw.find_all('img', class_='card-img')[0]['data-src']
            level = ''
            if '_bronze.' in level_raw:
                level = 'Bronze'
            elif '_silver.' in level_raw:
                level = 'Silver'
            elif '_gold.' in level_raw:
                level = 'Gold'
            elif 'hof.' in level_raw:
                level = 'Hall of Fame'

            badges[name] = {'Badge Category': category, 
                           'Badge Description': description, 
                           'Badge Level': level}
        
        per_player_dict['Badges'] = badges
        
        data[player_url] = per_player_dict
        print(per_player_dict['Team'] + '    ' + per_player_dict['Name'] + ' - Successfully scraped!')
        

df = pd.DataFrame.from_dict(data, orient='index')
df.sample(10)

Detroit Pistons Cade Cunningham 84Playmaking Shot Creator
Detroit Pistons Bojan Bogdanovic 83Sharpshooter
Detroit Pistons Jalen Duren 80Glass-Cleaning Finisher
Detroit Pistons Jaden Ivey 79Slashing Combo Guard
Detroit Pistons Isaiah Stewart 78Inside the Arc Scorer
Detroit Pistons Marvin Bagley III 78Inside the Arc Scorer
Detroit Pistons Alec Burks 77Sharpshooter
Detroit Pistons Hamidou Diallo 77Interior Finisher
Detroit Pistons James Wiseman 75Interior Finisher
Detroit Pistons Killian Hayes 75Playmaking Shot Creator
Detroit Pistons Nerlens Noel 752-Way Interior Finisher
Detroit Pistons Cory Joseph 73Slashing Playmaker
Detroit Pistons Isaiah Livers 72Stretch Four
Detroit Pistons Rodney McGruder 713 & D Two Guard
Detroit Pistons Buddy Boeheim 68Shooter
Detroit Pistons Stanley Umude 683PT Shooter
Detroit Pistons Jared Rhoden 67Perimeter Defender
Golden State Warriors Stephen Curry 96Versatile Offensive Force
Golden State Warriors Klay Thompson 862-Way 3-Level Scorer
Golden State Warriors 

Minnesota Timberwolves Luka Garza 73Glass Cleaner
Minnesota Timberwolves Wendell Moore Jr. 713PT Shooter
Minnesota Timberwolves Matt Ryan 713PT Specialist
Minnesota Timberwolves Josh Minott 70Prospect
New Orleans Pelicans Zion Williamson 92Slashing Playmaker
New Orleans Pelicans Brandon Ingram 86Playmaking Sharpshooter
New Orleans Pelicans C.J. McCollum 85Versatile Offensive Force
New Orleans Pelicans Jonas Valanciunas 83Inside-Out Glass Cleaner
New Orleans Pelicans Trey Murphy 79Stretch Four
New Orleans Pelicans Herbert Jones 78Hybrid Defender
New Orleans Pelicans Larry Nance Jr. 77Inside-Out Scorer
New Orleans Pelicans Jose Alvarado 772-Way Inside-Out Scorer
New Orleans Pelicans Josh Richardson 772-Way 3-Level Scorer
New Orleans Pelicans Naji Marshall 763-Level Scorer
New Orleans Pelicans Willy Hernangomez 76Glass Cleaner
New Orleans Pelicans Dyson Daniels 752-Way 3PT Shot Creator
New Orleans Pelicans Jaxson Hayes 752-Way Interior Finisher
New Orleans Pelicans Kira Lewis Jr. 72Tempo-

Toronto Raptors Gary Trent Jr. 803-Level Scorer
Toronto Raptors Jakob Poeltl 79Paint Beast
Toronto Raptors Chris Boucher 79Defensive Anchor
Toronto Raptors Precious Achiuwa 78Glass-Cleaning Finisher
Toronto Raptors Thaddeus Young 772-Way Interior Finisher
Toronto Raptors Otto Porter Jr. 763-Level Scorer
Toronto Raptors Dalano Banton 75Mid-Range Specialist
Toronto Raptors Christian Koloko 73Rim Protector
Toronto Raptors Malachi Flynn 73Inside-Out Scorer
Toronto Raptors Juancho Hernangomez 70Scrappy Wing
Toronto Raptors Joe Wieskamp 70Athletic Finisher
Toronto Raptors Ron Harper Jr. 683PT Shooter
Toronto Raptors Jeff Dowtin 673PT Shooter
Utah Jazz Lauri Markkanen 863-Level Scorer
Utah Jazz Jordan Clarkson 82Offensive Threat
Utah Jazz Collin Sexton 81Scoring Machine
Utah Jazz Walker Kessler 81Defensive Anchor
Utah Jazz Kelly Olynyk 78Inside-Out Scorer
Utah Jazz Talen Horton-Tucker 76Inside the Arc Scorer
Utah Jazz Rudy Gay 75Mid-Range Shot Creator
Utah Jazz Ochai Agbaji 743PT Shooter
Utah

,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Height,Weight,Jersey,...,Perimeter Defense,Steal,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges
https://www.2kratings.com/draymond-green,2023-02-17,Golden State Warriors,Draymond Green,83,United States,2-Way Slashing Playmaker,PF / C,"6'6"" (198cm)",230lbs (104kg),23,...,86,60,65,82,93,84,95,38,78,"{'Anchor': {'Badge Category': 'DEF/REB Badge',..."
https://www.2kratings.com/bam-adebayo,2023-02-17,Miami Heat,Bam Adebayo,88,United States,2-Way Inside-the-Arc Scorer,C,"6'9"" (206cm)",255lbs (115kg),13,...,83,52,60,73,92,80,90,66,85,"{'Anchor': {'Badge Category': 'DEF/REB Badge',..."
https://www.2kratings.com/brook-lopez,2023-02-17,Milwaukee Bucks,Brook Lopez,83,United States,2-Way 3-Level Scorer,C,"7'0"" (213cm)",282lbs (127kg),11,...,45,36,93,60,93,69,90,57,61,"{'Anchor': {'Badge Category': 'DEF/REB Badge',..."
https://www.2kratings.com/brandon-boston-jr,2023-02-17,Los Angeles Clippers,Brandon Boston Jr.,74,United States,3PT Shooter,SG / SF,"6'6"" (198cm)",188lbs (85kg),4,...,62,40,40,72,52,50,30,38,46,{}
https://www.2kratings.com/ron-harper-jr,2023-02-17,Toronto Raptors,Ron Harper Jr.,68,United States / Philippines,3PT Shooter,SG,"6'6"" (198cm)",245lbs (110kg),7,...,60,43,47,69,60,52,55,34,46,{}
https://www.2kratings.com/furkan-korkmaz,2023-02-17,Philadelphia 76ers,Furkan Korkmaz,71,Turkey,3PT Shooter,SF / SG,"6'7"" (201cm)",202lbs (91kg),30,...,64,41,51,69,56,62,45,37,53,{'Corner Specialist': {'Badge Category': 'Shoo...
https://www.2kratings.com/ziaire-williams,2023-02-17,Memphis Grizzlies,Ziaire Williams,72,United States,Inside the Arc Scorer,SF / PF,"6'9"" (206cm)",185lbs (83kg),8,...,73,42,51,77,70,71,65,31,68,{'Aerial Wizard': {'Badge Category': 'Finishin...
https://www.2kratings.com/jordan-mclaughlin,2023-02-17,Minnesota Timberwolves,Jordan McLaughlin,74,United States,Slashing Playmaker,PG / SG,"5'11"" (180cm)",170lbs (77kg),6,...,73,76,40,74,52,74,60,38,46,{}
https://www.2kratings.com/malik-monk,2023-02-17,Sacramento Kings,Malik Monk,79,United States,Inside the Arc Scorer,SG / PG,"6'3"" (191cm)",200lbs (90kg),0,...,68,41,37,73,69,68,45,35,47,{'Agent 3': {'Badge Category': 'Shooting Badge...
https://www.2kratings.com/daniel-theis,2023-02-17,Indiana Pacers,Daniel Theis,75,Germany,2-Way Inside-the-Arc Scorer,PF / C,"6'9"" (206cm)",245lbs (110kg),27,...,53,37,68,54,83,57,80,57,72,{'Brick Wall': {'Badge Category': 'DEF/REB Bad...


In [63]:
df = df.rename(columns={'Unnamed: 0': 'url'})
df = df.set_index('url')

df['Position'] = df['Position'].apply(lambda x: x.replace(' ', '').split('/'))

df['Weight'] = df['Weight'].str.extract(r'\((\d*)k')
df = df.rename(columns={'Weight': 'Weight (kg)'})

df['Height (cm)'] = df['Height'].str.extract(r'\((\d*)c')
df['Height (in)'] = df['Height'].str.extract(r'(\d.*")')

df

,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Height,Weight,Jersey,...,Perimeter Defense,Steal,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges
url,,,,,,,,,,,,,,,,,,,,,
https://www.2kratings.com/trae-young,2023-02-17,Atlanta Hawks,Trae Young,89,United States,Versatile Offensive Force,PG,"6'1"" (185cm)",180lbs (81kg),11.0,...,55,37,35,70,58,70,30,36,41,{'Agent 3': {'Badge Category': 'Shooting Badge...
https://www.2kratings.com/dejounte-murray,2023-02-17,Atlanta Hawks,Dejounte Murray,85,United States,2-Way Playshot,PG,"6'4"" (193cm)",180lbs (81kg),5.0,...,86,80,47,87,85,89,85,35,62,{'Ankle Braces': {'Badge Category': 'DEF/REB B...
https://www.2kratings.com/clint-capela,2023-02-17,Atlanta Hawks,Clint Capela,83,Switzerland,Paint Beast,C,"6'10"" (208cm)",256lbs (115kg),15.0,...,53,38,80,60,86,74,85,93,92,{'Post Move Lockdown': {'Badge Category': 'DEF...
https://www.2kratings.com/john-collins,2023-02-17,Atlanta Hawks,John Collins,82,United States,Inside the Arc Scorer,PF / C,"6'9"" (206cm)",226lbs (102kg),20.0,...,49,40,76,57,74,38,60,45,85,{'Aerial Wizard': {'Badge Category': 'Finishin...
https://www.2kratings.com/onyeka-okongwu,2023-02-17,Atlanta Hawks,Onyeka Okongwu,80,United States,2-Way Inside-the-Arc Scorer,C / PF,"6'8"" (203cm)",240lbs (108kg),17.0,...,64,40,80,69,83,65,65,81,78,{'Aerial Wizard': {'Badge Category': 'Finishin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.2kratings.com/taj-gibson,2023-02-17,Washington Wizards,Taj Gibson,73,United States,Inside the Arc Scorer,C,"6'9"" (206cm)",232lbs (104kg),67.0,...,58,50,65,57,78,62,60,58,56,{'Vice Grip': {'Badge Category': 'Playmaking B...
https://www.2kratings.com/johnny-davis,2023-02-17,Washington Wizards,Johnny Davis,72,United States,Mid-Range Shooter,SG,"6'5"" (196cm)",195lbs (88kg),1.0,...,73,46,37,79,68,55,60,36,38,{}
https://www.2kratings.com/anthony-gill,2023-02-17,Washington Wizards,Anthony Gill,72,United States,Interior Finisher,PF,"6'7"" (201cm)",230lbs (104kg),16.0,...,60,35,53,50,50,58,50,53,48,{}


In [69]:
df.dtypes

scrape_date               object
Team                      object
Name                      object
Overall Rating             int64
Nationality               object
Archetype                 object
Position                  object
Height                    object
Weight (kg)               object
Jersey                   float64
Years in NBA             float64
Prior to NBA              object
Close Shot                 int64
Mid-Range Shot             int64
Three-Point Shot           int64
Free Throw                 int64
Shot IQ                    int64
Offensive Consistency      int64
Speed                      int64
Acceleration               int64
Strength                   int64
Vertical                   int64
Stamina                    int64
Hustle                     int64
Overall Durability         int64
Layup                      int64
Standing Dunk              int64
Driving Dunk               int64
Post Hook                  int64
Post Fade                  int64
Post Contr

In [91]:
# df[df['Jersey'].isnull()]
df['Jersey']['https://www.2kratings.com/tyty-washington'] = '0'
df.loc[['https://www.2kratings.com/tyty-washington']]

C:\Users\BUDDY\AppData\Local\Temp\ipykernel_1920\3201676779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Jersey']['https://www.2kratings.com/tyty-washington'] = '0'


,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Height,Weight (kg),Jersey,...,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges,Height (cm),Height (in)
url,,,,,,,,,,,,,,,,,,,,,
https://www.2kratings.com/tyty-washington,2023-02-17,Houston Rockets,TyTy Washington,72,United States,Mid-Range Shot Creator,[PG],"6'3"" (191cm)",88,0,...,25,76,62,70,40,29,58,{},191,"6'3"""


In [92]:
# df[df['Years in NBA'].isnull()]
df['Years in NBA']['https://www.2kratings.com/anthony-lamb'] = '2'
df['Years in NBA']['https://www.2kratings.com/trevelin-queen'] = '1'
df['Years in NBA']['https://www.2kratings.com/davon-reed'] = '3'
df['Years in NBA']['https://www.2kratings.com/omer-yurtseven'] = '1'
df['Years in NBA']['https://www.2kratings.com/kevon-harris'] = '0'
df['Years in NBA']['https://www.2kratings.com/jeff-dowtin'] = '1'

,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Height,Weight (kg),Jersey,...,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges,Height (cm),Height (in)
url,,,,,,,,,,,,,,,,,,,,,
https://www.2kratings.com/anthony-lamb,2023-02-17,Golden State Warriors,Anthony Lamb,74,United States,3PT Specialist,"[SF, SG]","6'6"" (198cm)",102,33.0,...,52,67,59,50,35,47,61,{},198,"6'6"""
https://www.2kratings.com/trevelin-queen,2023-02-17,Indiana Pacers,Trevelin Queen,67,United States,Scrappy Two Guard,[SG],"6'6"" (198cm)",86,9.0,...,51,70,55,47,40,54,64,{},198,"6'6"""
https://www.2kratings.com/davon-reed,2023-02-17,Los Angeles Lakers,Davon Reed,71,United States,Shooter,[SG],"6'5"" (196cm)",93,9.0,...,45,69,60,55,40,42,60,{},196,"6'5"""
https://www.2kratings.com/omer-yurtseven,2023-02-17,Miami Heat,Omer Yurtseven,75,Turkey,Glass Cleaner,[C],"6'11"" (211cm)",124,77.0,...,70,49,62,58,40,89,90,{},211,"6'11"""
https://www.2kratings.com/kevon-harris,2023-02-17,Orlando Magic,Kevon Harris,69,United States,3PT Shooter,[SG],"6'6"" (198cm)",97,7.0,...,47,64,45,60,45,63,50,{},198,"6'6"""
https://www.2kratings.com/jeff-dowtin,2023-02-17,Toronto Raptors,Jeff Dowtin,67,United States,3PT Shooter,[PG],"6'3"" (191cm)",80,15.0,...,50,70,58,52,40,66,52,{},191,"6'3"""


In [97]:
# df[df.isna().any(axis=1)]
df['Nationality']['https://www.2kratings.com/jordan-goodwin'] = 'United States'
df['Nationality']['https://www.2kratings.com/quenton-jackson'] = 'United States'

,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Height,Weight (kg),Jersey,...,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges,Height (cm),Height (in)
url,,,,,,,,,,,,,,,,,,,,,
https://www.2kratings.com/jordan-goodwin,2023-02-17,Washington Wizards,Jordan Goodwin,74,NaN,3-Level Scorer,"[SG, PG]","6'3"" (191cm)",90,7.0,...,63,71,62,63,50,57,63,{},191,"6'3"""
https://www.2kratings.com/quenton-jackson,2023-02-17,Washington Wizards,Quenton Jackson,68,NaN,Perimeter Defender,"[SG, PG]","6'5"" (196cm)",79,29.0,...,40,78,70,75,65,46,42,{},196,"6'5"""


In [104]:
df['Nationality'] = df['Nationality'].apply(lambda x: x.replace('  /  ', '@').split('@'))
df = df.astype({'Height (cm)': 'int64', 'Jersey': 'int64', 'Years in NBA': 'int64', 'Weight (kg)': 'int64'})
df = df.drop(columns=['Height'])
df.dtypes

scrape_date              object
Team                     object
Name                     object
Overall Rating            int64
Nationality              object
Archetype                object
Position                 object
Weight (kg)               int64
Jersey                    int64
Years in NBA              int64
Prior to NBA             object
Close Shot                int64
Mid-Range Shot            int64
Three-Point Shot          int64
Free Throw                int64
Shot IQ                   int64
Offensive Consistency     int64
Speed                     int64
Acceleration              int64
Strength                  int64
Vertical                  int64
Stamina                   int64
Hustle                    int64
Overall Durability        int64
Layup                     int64
Standing Dunk             int64
Driving Dunk              int64
Post Hook                 int64
Post Fade                 int64
Post Control              int64
Draw Foul                 int64
Hands   

In [111]:
df.to_csv('nba2k23_data.csv')